In [ ]:
import PIL
import numpy as np
import torch

from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

# device = "cuda"
# pipeline = AutoPipelineForInpainting.from_pretrained(
#     "runwayml/stable-diffusion-inpainting",
#     torch_dtype=torch.float16,
# )
# pipeline = pipeline.to(device)



In [ ]:
from diffusers.pipelines import StableDiffusionInpaintPipeline
pipeline  = StableDiffusionInpaintPipeline.from_pretrained(
            "stabilityai/stable-diffusion-2-inpainting",
            torch_dtype=torch.float16,
        ).to("cuda")


In [ ]:
from PIL import Image
import cv2
img_url = "/home/tlips/Documents/diffusing-synthetic-data/data/diffusion_renders/mugs/run_9/4a2fe98254114412b73b6b831b1a38a4/000/CropAndInpaintRenderer/A light yellow  mug with a glazed finish, Photorealistic+a bamboo table with a natural, sustainable feel_0.png"
mask_url = "/home/tlips/Documents/diffusing-synthetic-data/data/diffusion_renders/mugs/run_9/4a2fe98254114412b73b6b831b1a38a4/000/original/segmentation.png"

init_image = load_image(img_url).resize((512, 512))
mask_image = load_image(mask_url).resize((512, 512))
original_mask_image = mask_image
mask_image = 1.0*(np.array(mask_image) > 0)
mask_image = cv2.dilate(mask_image, np.ones((5,5), np.uint8), iterations=1)

mask_image = 1- mask_image * 1.0
mask_image = Image.fromarray((mask_image*255).astype(np.uint8))


prompt = "a black wooden table"
repainted_image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image,num_images_per_prompt=2).images[0]
#repainted_image.save("repainted_image.png")

unmasked_unchanged_image = pipeline.image_processor.apply_overlay(original_mask_image, repainted_image,init_image)
#unmasked_unchanged_image = Image.blend(unmasked_unchanged_image,original_mask_image, 0.5)
#unmasked_unchanged_image.ave("force_unmasked_unchanged.png")
make_image_grid([init_image, mask_image, repainted_image, unmasked_unchanged_image], rows=2, cols=2)